<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


1. 네이버 자료

2. 네이버 Place 코드
    * 기준: 서울시 동별, 업종별
    
    
3. 가게정보, 리뷰 정도 겟



# 필요함수 및 라이브러리

In [1]:
import time
import pandas as pd
import numpy as np
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action = 'ignore')


## 파일 관련

In [2]:
def get_today():
    import datetime
    now = datetime.datetime.now()
    
    nowDate = now.strftime('%Y-%m-%d')
#     print(nowDate)      # 2015-04-19
 
    nowTime = now.strftime('%H:%M:%S')
#     print(nowTime)      # 12:11:32

    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    return nowDate

In [3]:
import os
def check_file(file_path, file_name):
    return os.path.isfile(file_path+file_name)

In [4]:
def save_df(df, path, file_name):
    df.to_csv(path+file_name, index=False)
    print(f'{path+file_name} 저장 완료')

In [5]:
def read_csv_코드정보(code_path , code_file):
    df = pd.read_csv(code_path + code_file )
    return df

In [6]:
'''음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,
   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, 
   영화공연:CINEMA, 관공서:GOVERNMENT'''

'음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,\n   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, \n   영화공연:CINEMA, 관공서:GOVERNMENT'

###### 서울시 동 load 관련

In [7]:
import pickle
import collections

def get_seoul_dong():
    with open('data/서울시_구별_동정리.pickle','rb') as fw:
        dong_dict = pickle.load( fw)

    dong_key_list = list(dong_dict.keys())
    dong_key_list.sort()
    # dong_key_list

    for 구이름 in dong_key_list:
        dong_dict[구이름] = sorted(dong_dict[구이름])
    
    return dong_dict

In [8]:
dong_dict = get_seoul_dong()

## 웹 조정 관련

In [9]:
def is_clickable(wd):
    return wd.is_enabled() and wd.is_displayed()

In [10]:
def scroll_down(wd,SCROLL_PAUSE_SEC = 1):

    # 스크롤 높이 가져옴
    last_height = wd.execute_script("return document.body.scrollHeight")
#     print(last_height)
    while True:
        # 끝까지 스크롤 다운
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

## 크롤링 에러 방지 관련

In [11]:
def try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        return wd.find_element_by_class_name(class_name).text
    except NoSuchElementException:
        print(col, end=' ')
        return np.NaN
        

In [12]:
def strong_try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        temps = wd.find_elements_by_class_name(class_name)
        for temp in temps:
            if temp.text.startswith('편의'):
                return temp.text.split('\n')[1]
        else:
            return np.NaN
    except NoSuchElementException:
#         print(col, end='')
        return np.NaN
        

In [13]:
def review_dict(s):
    review_dict = {}
#     print(s.split('평균 별점'))
    s = s.split('평균 별점')
    review_dict['평균_별점'] = s[-1].strip()
    s = s[0].split('사진')
    if len(s) >= 2:
        review_dict['사진'] = int(s[-1])
        review_dict['리뷰'] = s[0].split()[-1]
    else:
        review_dict['사진'] = np.NaN
        review_dict['리뷰'] = s[0].split()[-1]
# print(s)
    return review_dict

## get_store_info from naver.place by place_code

In [14]:
def get_store_info(wd, code_no):
    name = wd.find_element_by_class_name('_3XamX').text
#     print(name)
    r_type = try_none_text_by_class_name(wd,'_3ocDE','ty')
#     print(r_type)
    address = try_none_text_by_class_name(wd,'_2yqUQ','ad')
#     print(address)
    tel = try_none_text_by_class_name(wd,'_3ZA0S','tel') 
#     print(tel)
    utility, detail = np.NaN, np.NaN
    for i in wd.find_elements_by_class_name('_1M_Iz'):
#         print(i.text)
        if '편의' in i.text :
            utility = i.text.replace('편의\n','')
        elif '설명' in i.text :
            detail = i.text.replace('설명\n', '')
    
#     print(code_no, utility, detail)
    
    nums_list = wd.find_elements_by_class_name('_1Y6hi')
    try:
        score = nums_list[0].find_element_by_tag_name('em').text
    except:
        score = np.NaN
    try:
        review_num = nums_list[1].find_element_by_tag_name('em').text
    except:
        review_num = np.NaN
        
    try:
        blog_nums = nums_list[2].find_element_by_tag_name('em').text
    except:
        blog_nums = np.NaN
#     print(score, review_num, blog_nums)
    collected_date = get_today()

    # 데이터 렙 펼치기
    for i in wd.find_elements_by_class_name('_3iTUo'):
        if i.text=='더보기':
            i.send_keys('\n')
        else:
            pass
            
    try:
        datalab_1 = {}
        datalab_list = wd.find_elements_by_class_name('_3Ryhx')
        for datalab in datalab_list:
            datalab_1[datalab.find_element_by_class_name('_3hvd9').text] = [x.text for x in datalab.find_elements_by_class_name('_2irYJ') if x.text != '']
    except:
        datalab_1 = np.NaN
        print('d_1', end='')
#     print(name, datalab_1)

    try:
        datalab_g = {}
        datalab_block = wd.find_element_by_class_name('_250cA')
        datalab_age_list = datalab_block.find_elements_by_class_name('_1Nyv0')
        for age_info in datalab_age_list:
            age = age_info.find_element_by_class_name('iO1iA').text
            preference = age_info.find_element_by_class_name('place_blind').text.replace('%','')
            datalab_g[age] = preference
            
        gender_preferance = wd.find_element_by_class_name('c3-chart-arc.c3-target.c3-target-female').text.replace('%',"")
        datalab_g['female_pref'] = gender_preferance
    except:
        datalab_g = np.NaN
        print('d_g', end="")
#     print(datalab_1, datalab_g)

    return name, code_no, r_type, address,tel, score, utility, detail, datalab_1, datalab_g, review_num, blog_nums, collected_date

In [15]:
code_path = 'data/코드/'
구이름 = '강남구'
업종 = 'DINING'
code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'

In [16]:
code_df = read_csv_코드정보(code_path, code_file)

In [17]:
code_df.shape

(6863, 2)

In [18]:
def naver_map_store_scrapping( 구이름, 업종, slow= 1):
    start_time = time.time()

    options = webdriver.ChromeOptions()

    options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(1*slow)
    
    code_path = 'data/코드/'
    review_path = 'data/리뷰/'
    store_path = 'data/가게/'

    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    review_file = '리뷰정보_' + 구이름 + '_'+ 업종 + '.csv'
    store_file = '가게정보_' + 구이름 + '_'+ 업종 + '.csv'
    store_idx = 0
    store_df = pd.DataFrame(columns=("name", "code_no", "r_type","address","tel",
                                     "score","utility","detail","datalab_1", "datalab_g", "review_num",
                                     "blog_num", "collected_date"))

    current_num = 1

    code_df = pd.read_csv(code_path+code_file, dtype='str')

    print(f'{구이름}  {업종} 업종 의 가게 정보 스크래핑을 시작합니다.')

    print(f'{code_df.shape[0]}개의 가게코드가 있습니다.')

    for _, code_no_ in code_df.iterrows():
        code_no = code_no_.values[1]

        place_url = 'https://pcmap.place.naver.com/restaurant/'+str(code_no)

        wd.get(place_url)
        time.sleep(1*slow)
        scroll_down(wd,0.5)
        cnt_idx = 0

        try:
#             input('ddd')
            wd.implicitly_wait(0.001*slow)
            store_df.loc[store_idx] = get_store_info(wd,code_no)
            wd.implicitly_wait(1*slow)
            print(store_idx, end='#')
            store_idx += 1
            time.sleep(5)
            
        except:
            print(store_idx, end='!!')
            pass
        
        ##################### cooling down
        if store_idx%100 == 0 :
            wd.close()
            wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
            wd.implicitly_wait(1*slow)
            time.sleep(1*slow)
        ##################### Test Test Test
#         if store_idx >=10:
#             store_df.to_csv(store_path+store_file, index=False)
#             print(구이름,업종, f'가게 {store_idx}개 완료' )
#             print(f'{time.time()-start_time} 소요')
#             wd.close()
#             break
    else:
        store_df.to_csv(store_path+store_file, index=False)
        print(구이름, 업종, f'가게 {store_idx}개 완료' )
        print(f'{time.time()-start_time} 소요')
        wd.close()
        
#     return store_df

In [18]:
def naver_map_store_scrapping( 구이름, 업종, slow= 1):
    start_time = time.time()

    options = webdriver.ChromeOptions()

    options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(1*slow)
    
    code_path = 'data/코드/'
    review_path = 'data/리뷰/'
    store_path = 'data/가게/'

    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    review_file = '리뷰정보_' + 구이름 + '_'+ 업종 + '.csv'
    store_file = '가게정보_' + 구이름 + '_'+ 업종 + '.csv'
    
    store_idx = 0
    store_df = pd.DataFrame(columns=("name", "code_no", "r_type","address","tel",
                                     "score","utility","detail","datalab_1", "datalab_g", "review_num",
                                     "blog_num", "collected_date"))
    
    if check_file(store_path, store_file):
        store_df = pd.read_csv(store_path+store_file)
        store_idx = store_df.shape[0]
    done_list = store_df['code_no'].value.to_list()
    code_df = pd.read_csv(code_path+code_file, dtype='str')

    print(f'{구이름}  {업종} 업종 의 가게 정보 스크래핑을 시작합니다.')

    print(f'{code_df.shape[0]}개의 가게코드가 있습니다.')
    
    print(f'{store_idx}가 이전에 완료 되었었습니다. 이어서 진행합니다.')

    for _, code_no_ in code_df.iterrows():
        code_no = code_no_.values[1]
        if code_no in done_list:
            print(code_no)
            continue
            
        place_url = 'https://pcmap.place.naver.com/restaurant/'+str(code_no)

        wd.get(place_url)
        time.sleep(1*slow)
        scroll_down(wd,0.5)
        cnt_idx = 0

        try:
#             input('ddd')
            wd.implicitly_wait(0.001*slow)
            store_df.loc[store_idx] = get_store_info(wd,code_no)
            wd.implicitly_wait(1*slow)
            print(store_idx, end='#')
            store_idx += 1
            time.sleep(5)
            
        except:
            print(store_idx, end='!!')
            pass
        
        ##################### cooling down
        if store_idx%100 == 0 :
            wd.close()
            wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
            wd.implicitly_wait(1*slow)
            time.sleep(1*slow)
            
        ##################### save just in case:
        if store_idx%500 == 0 :
            store_df.to_csv(store_path+store_file, index=False)
            print(구이름, 업종, f'가게 {store_idx}개 중간저장')
            print(f'{time.time()-start_time} 소요')
        ##################### Test Test Test
#         if store_idx >=10:
#             store_df.to_csv(store_path+store_file, index=False)
#             print(구이름,업종, f'가게 {store_idx}개 완료' )
#             print(f'{time.time()-start_time} 소요')
#             wd.close()
#             break
    else:
        store_df.to_csv(store_path+store_file, index=False)
        print(구이름, 업종, f'가게 {store_idx}개 완료' )
        print(f'{time.time()-start_time} 소요')
        wd.close()
        
#     return store_df

# 셋팅

In [19]:
# code_path = 'data/코드/'
# review_path = 'data/리뷰/'
# store_path = 'data/가게/'

# code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'

In [20]:
code_path = 'data/코드/'
store_path = 'data/가게/'
# code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 +  '_'+ 업종 + '.csv'

In [21]:
detail= True
slow=1

#  코드 수집 및 저장

In [22]:
업종= 'DINING'

for 구이름 in sorted(dong_dict.keys()):
    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    if check_file(code_path, code_file):
        store_file = '가게정보_' + 구이름 +  '_'+ 업종 + '.csv'
        if check_file(store_path, store_file):
            print(f'{구이름} 가게 정보 수집 완료')
            pass
        else:
            naver_map_store_scrapping( 구이름, 업종, slow= 1)
    else:
        print(f'{구이름} {업종}이 없습니다. ')

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


강남구 가게 정보 수집 완료
강동구 DINING이 없습니다. 
강북구 DINING이 없습니다. 
강서구 DINING이 없습니다. 
관악구 DINING이 없습니다. 
광진구 DINING이 없습니다. 
구로구 DINING이 없습니다. 
금천구 DINING이 없습니다. 
노원구 DINING이 없습니다. 
도봉구 DINING이 없습니다. 
동대문구 DINING이 없습니다. 
동작구 DINING이 없습니다. 
마포구 DINING이 없습니다. 
서대문구 DINING이 없습니다. 
서초구 DINING이 없습니다. 
성동구 DINING이 없습니다. 
성북구 DINING이 없습니다. 
송파구 DINING이 없습니다. 
양천구 DINING이 없습니다. 
영등포구 DINING이 없습니다. 
용산구 DINING이 없습니다. 
은평구  DINING 업종 의 가게 정보 스크래핑을 시작합니다.
3261개의 가게코드가 있습니다.
0#1#2#3#4#5#6#7#8#9#10#11#12#13#14#15#16#17#18#19#20#21#22#23#24#25#26#27#28#29#30#31#32#33#34#35#36#37#38#39#40#41#42#43#44#45#46#47#48#49#50#51#52#53#54#55#56#57#58#59#60#61#62#63#64#d_g65#66#67#68#69#70#71#72#73#74#75#76#77#78#79#80#d_g81#82#83#d_g84#d_g85#86#87#d_g88#89#90#d_g91#92#d_g93#d_g94#d_g95#d_g96#d_g97#98#99#

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


100#d_g101#d_g102#d_g103#104#d_g105#d_g106#d_g107#108#d_g109#d_g110#d_g111#112#d_g113#114#d_g115#d_g116#117#d_g118#119#120#121#122#123#d_g124#125#d_g126#127#d_g128#d_g129#d_g130#131#d_g132#d_g133#d_g134#d_g135#136#137#d_g138#d_g139#d_g140#d_g141#d_g142#d_g143#d_g144#d_g145#d_g146#d_g147#148#d_g149#tel d_g150#d_g151#tel d_g152#tel d_g153#154#tel d_g155#156#d_g157#158#159#160#161#162#163#164#165#d_g166#d_g167#d_g168#d_g169#d_g170#d_g171#d_g172#d_g173#d_g174#d_g175#d_g176#d_g177#d_g178#d_g179#d_g180#d_g181#d_g182#d_g183#tel d_g184#d_g185#186#187#188#189#190#d_g191#d_g192#d_g193#194#d_g195#d_g196#197#d_g198#d_g199#

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


...

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


d_g400#d_g401#d_g402#d_g403#d_g404#d_g405#d_g406#d_g407#d_g408#d_g409#d_g410#d_g411#d_g412#d_g413#d_g414#d_g415#d_g416#d_g417#d_g418#419#d_g420#d_g421#422#d_g423#d_g424#425#d_g426#d_g427#d_g428#d_g429#d_g430#d_g431#d_g432#d_g433#d_g434#d_g435#d_g436#d_g437#d_g438#d_g439#d_g440#d_g441#d_g442#443#d_g444#d_g445#d_g446#d_g447#d_g448#d_g449#d_g450#d_g451#d_g452#d_g453#d_g454#d_g455#d_g456#d_g457#d_g458#d_g459#d_g460#d_g461#d_g462#d_g463#d_g464#d_g465#tel d_g466#d_g467#d_g468#d_g469#d_g470#471#d_g472#d_g473#474#d_g475#d_g476#d_g477#d_g478#d_g479#d_g480#d_g481#d_g482#d_g483#d_g484#d_g485#486#d_g487#d_g488#d_g489#d_g490#d_g491#d_g492#d_g493#d_g494#d_g495#d_g496#d_g497#d_g498#d_g499#

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.17/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.17]


d_g500#d_g501#502#d_g503#d_g504#d_g505#d_g506#d_g507#508#d_g509#d_g510#d_g511#d_g512#d_g513#d_g514#d_g515#d_g516#d_g517#518#d_g519#520#d_g521#d_g522#d_g523#d_g524#d_g525#d_g526#d_g527#d_g528#d_g529#d_g530#d_g531#d_g532#d_g533#534#d_g535#d_g536#d_g537#d_g538#d_g539#d_g540#d_g541#d_g542#d_g543#d_g544#d_g545#d_g546#d_g547#d_g548#d_g549#d_g550#d_g551#d_g552#d_g553#554#555#d_g556#d_g557#d_g558#d_g559#d_g560#d_g561#d_g562#d_g563#

WebDriverException: Message: chrome not reachable
  (Session info: chrome=95.0.4638.54)


In [ ]:
df = pd.read_csv('data/가게/가게정보_중랑구_DINING.csv')

In [ ]:
df